In [103]:
# export CMAKE_ARGS="-DLLAMA_METAL=ON \
#   -DLLAMA_METAL_EMBED_LIBRARY=ON \
#   -DCMAKE_BUILD_TYPE=Release \
#   -DLLAMA_NATIVE=ON \
#   -DLLAMA_ACCELERATE=ON \
#   -DCMAKE_C_FLAGS='-O3 -march=native -mtune=native -ffast-math' \
#   -DCMAKE_CXX_FLAGS='-O3 -march=native -mtune=native -ffast-math'"

# %pip install llama-cpp-python --force-reinstall --no-cache-dir

# LSP Client

In [104]:
import json
import subprocess
import tempfile
import os
import time
from typing import List, Optional


class LSPClient:
    def __init__(self, cmd: List[str]):
        """Start LSP server with given command (e.g., ['pylsp'])"""
        self.process = subprocess.Popen(
            cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE, text=True, bufsize=0
        )
        self.req_id = 0
        time.sleep(0.1)  # Give server time to start
        self._init_lsp()
        
    def _init_lsp(self):
        """Initialize LSP connection"""
        response = self._send_request("initialize", {
            "processId": os.getpid(),
            "capabilities": {
                "textDocument": {
                    "completion": {"completionItem": {"snippetSupport": False}}
                }
            }
        })
        
        if response:
            self._send_notification("initialized", {})
    
    def _read_message(self) -> Optional[dict]:
        """Read one LSP message"""
        try:
            # Read headers
            content_length = 0
            while True:
                line = self.process.stdout.readline()
                if not line:
                    return None
                if line.startswith("Content-Length:"):
                    content_length = int(line.split(":")[1].strip())
                elif line.strip() == "":
                    break  # End of headers
            
            if content_length == 0:
                return None
                
            # Read content
            content = self.process.stdout.read(content_length)
            return json.loads(content)
        except:
            return None
    
    def _send_notification(self, method: str, params: dict):
        """Send notification (no response expected)"""
        msg = json.dumps({
            "jsonrpc": "2.0", 
            "method": method, 
            "params": params
        })
        full_msg = f"Content-Length: {len(msg)}\r\n\r\n{msg}"
        self.process.stdin.write(full_msg)
        self.process.stdin.flush()
    
    def _send_request(self, method: str, params: dict) -> Optional[dict]:
        """Send request and wait for response"""
        self.req_id += 1
        request_id = self.req_id
        
        msg = json.dumps({
            "jsonrpc": "2.0", 
            "id": request_id, 
            "method": method, 
            "params": params
        })
        
        full_msg = f"Content-Length: {len(msg)}\r\n\r\n{msg}"
        self.process.stdin.write(full_msg)
        self.process.stdin.flush()
        
        # Read messages until we get our response
        for _ in range(10):  # Max 10 attempts
            message = self._read_message()
            if message and message.get("id") == request_id:
                return message
            time.sleep(0.01)  # Small delay
        
        return None
    
    def _collect_diagnostics(self, uri: str, timeout: float = 1.0) -> list:
        """Collect diagnostics for a given file URI (from publishDiagnostics)."""
        diagnostics = []
        start = time.time()

        while time.time() - start < timeout:
            msg = self._read_message()
            if not msg:
                continue

            # Only care about publishDiagnostics notifications
            if msg.get("method") == "textDocument/publishDiagnostics":
                params = msg.get("params", {})
                if params.get("uri") == uri:
                    diagnostics.extend(params.get("diagnostics", []))
                    # Most LSPs send one diagnostics batch per change — can break early
                    break

            # Give other messages a chance to arrive
            time.sleep(0.01)

        return diagnostics
    
    def validate_code(self, code: str) -> bool:        
        
        # Create temp file
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(code)
            uri = f"file://{f.name}"
        
        try:
            # Open document in LSP
            self._send_notification("textDocument/didOpen", {
                "textDocument": {
                    "uri": uri,
                    "languageId": "python",
                    "version": 1,
                    "text": code
                }
            })
            
            # Collect diagnostics
            diagnostics = self._collect_diagnostics(uri)
            
            # Close document
            self._send_notification("textDocument/didClose", {
                "textDocument": {"uri": uri}
            })
            
            # Analyze diagnostics
            semantic_errors = []
            for d in diagnostics:
                msg = d.get("message", "").lower()
                
                # Ignore incomplete syntax or formatting issues
                if any(s in msg for s in [
                    "unexpected eof",
                    "was never closed",
                    "expected",
                    "newline at end",
                    "indentation",
                ]):
                    continue
                
                # Flag semantic problems
                if any(s in msg for s in [
                    "undefined name",
                    "attributeerror",
                    "keyerror",
                    "name is not defined",
                    "object has no attribute",
                    "not callable",
                    "cannot import",
                ]):
                    semantic_errors.append({
                        'message': d.get('message'),
                        'severity': d.get('severity'),
                        'range': d.get('range')
                    })
            
            
            # Determine if valid
            is_valid = len(semantic_errors) == 0  

            # return {
            #     'is_valid': is_valid,
            #     'match': match,
            #     'diagnostics': diagnostics,
            #     'semantic_errors': semantic_errors
            # }          
            
            return is_valid
            
        finally:
            os.unlink(f.name)
    
    def close(self):
        """Cleanup"""
        try:
            self._send_request("shutdown", {})
            self._send_notification("exit", {})
        except:
            pass
        
        self.process.terminate()
        self.process.wait(timeout=2)


# Small Language Model

In [105]:
from llama_cpp import Llama
from llama_cpp.llama_speculative import LlamaPromptLookupDecoding
import math

class LocalCodeModel:
    def __init__(
        self, 
        model_path: str,
        # Model configuration parameters
        n_ctx: int = 2048,
        n_gpu_layers: int = -1,
        n_threads: int = 4,
        n_batch: int = 1024,
        verbose: bool = False,
        use_mlock: bool = True,
        use_mmap: bool = True,
        logits_all: bool = True,
        # Generation default parameters
        max_tokens: int = 1,
        temperature: float = 0,
        top_p: float = 1,
        top_k: int = 1,
        stop_tokens: List[str] = []
    ):

        self.model = Llama(
            model_path=model_path,
            n_ctx=n_ctx,
            n_gpu_layers=n_gpu_layers,
            n_threads=n_threads,
            n_batch=n_batch,
            verbose=verbose,
            use_mlock=use_mlock,
            use_mmap=use_mmap,
            logits_all=logits_all,            
            # draft_model=LlamaPromptLookupDecoding(num_pred_tokens=max_tokens)
        )

        self.max_tokens = max_tokens
        self.temperature = temperature
        self.top_p = top_p
        self.top_k = top_k
        self.stop_tokens = stop_tokens
        self.n_ctx = n_ctx
        self.token_budget = self.calculate_token_budget()
        
    def calculate_token_budget(self):
        # Reserve tokens for generation
        reserved_for_generation = self.max_tokens
        
        # Reserve for special FIM tokens: <|fim_prefix|>, <|fim_suffix|>, <|fim_middle|>
        reserved_for_special_tokens = 30  # Conservative estimate
        
        # Safety buffer
        safety_buffer = 10
        
        # Calculate available tokens for input
        available_input_tokens = (
            self.n_ctx - 
            reserved_for_generation - 
            reserved_for_special_tokens - 
            safety_buffer
        )
        
        # Estimate chars per token (conservative for code)
        chars_per_token = 2.5
        
        # Calculate available characters
        available_chars = int(available_input_tokens * chars_per_token)
        print(f"Token budget: {available_input_tokens} tokens (~{available_chars} chars) available for input")

        return available_chars

    def truncate_context(
        self, 
        code_before: str, 
        code_after: str,
        split_ratio: float = 0.75
    ) -> tuple[str, str]:
        """
        Truncate code_before and code_after to fit within token budget.
        
        Args:
            code_before: Code before the cursor
            code_after: Code after the cursor
            split_ratio: Ratio of budget to allocate to code_before (0.75 = 75% before, 25% after)
        
        Returns:
            Tuple of (truncated_code_before, truncated_code_after)
        """
        # Calculate max characters for each part
        max_before_chars = int(self.token_budget * split_ratio)
        max_after_chars = int(self.token_budget * (1 - split_ratio))
        
        # Truncate: keep END of before (most recent), START of after (immediate next)
        truncated_before = code_before[-max_before_chars:] if len(code_before) > max_before_chars else code_before
        truncated_after = code_after[:max_after_chars] if len(code_after) > max_after_chars else code_after
        
        return truncated_before, truncated_after

    def build_fim_prompt(self, code_before: str, code_after: str) -> str:
        """Build FIM prompt from code context."""
        # return f"<|fim_begin|>{code_before}<|fim_hole|>{code_after}<|fim_end|>"
        return f"<|fim_prefix|>{code_before}<|fim_suffix|>{code_after}<|fim_middle|>"
    
    def clean_completion(self, completion: str, stop_chars: list[str]) -> str:
        """Remove trailing stop characters that slipped through."""
        cleaned = completion
        
        # Strip any stop characters from the end
        while cleaned and cleaned[-1] in stop_chars:
            cleaned = cleaned[:-1]
        
        return cleaned
    
    def generate(self, code_before: str, code_after: str):
        
        trunc_code_before, trunc_code_after = self.truncate_context(code_before=code_before, code_after=code_after)
        prompt = self.build_fim_prompt(code_before=trunc_code_before, code_after=trunc_code_after)

        output = self.model(
            prompt=prompt,
            max_tokens=self.max_tokens,       
            temperature=self.temperature,      
            top_p=self.top_p,            
            top_k=self.top_k,          
            stop=self.stop_tokens,
            logprobs=1
        )
       
        STOP_CHARS = ['(', ')', '[', ']', '{', '}', ',', ':', ';', '.']

        completions_w_probs = []
        for choice in output['choices']:
            completion = choice['text'].strip()
            
            if completion:
                cleaned = self.clean_completion(completion, STOP_CHARS)
                
                if cleaned:
                    # Simple: just get first token's probability
                    try:
                        first_logprob = choice['logprobs']['token_logprobs'][0]
                        if first_logprob is not None:
                            probability = math.exp(first_logprob)
                        else:
                            probability = 1.0
                    except (KeyError, IndexError, TypeError):
                        probability = 1.0  # Fallback
                    
                    
                    completions_w_probs.append((cleaned, probability))
        
            # Remove duplicates, keep highest probability
            seen = {}
            for text, prob in completions_w_probs:
                if text not in seen or prob > seen[text]:
                    seen[text] = prob
            
            result = list(seen.items())
            result.sort(key=lambda x: x[1], reverse=True)
            
        return result

# Loading data

In [106]:
import os
import json
from tokenize import tokenize, ENCODING, ENDMARKER, COMMENT
from io import BytesIO

class DataLoader:
    def __init__(self, basedir, infile, outfile):
        self.basedir = basedir
        self.infile = infile
        self.outfile = outfile
        self.tokens = []
    
    def tokenize_data(self):
        file_paths = open(os.path.join(self.basedir, self.infile)).readlines()
        files_with_tokens = []
        for ct, path in enumerate(file_paths):
            try:
                # print(path)
                code = open(os.path.join(self.basedir, path.strip())).read()
                token_gen = tokenize(BytesIO(bytes(code, "utf8")).readline)
                
                file_tokens = []
                for toknum, tokval, start, end, line in token_gen:
                    tokval = " ".join(tokval.split())
                    
                    if toknum in [ENCODING, ENDMARKER, COMMENT] or len(tokval) == 0:
                        continue
                    
                    start_line, start_col = start
                    
                    # Convert (line, col) to character position
                    lines = code.split('\n')
                    char_pos = sum(len(l) + 1 for l in lines[:start_line-1])
                    char_pos += start_col
                    
                    file_tokens.append({
                        'value': tokval,
                        'start_pos': char_pos,
                        'type': toknum
                    })

                    # print(f"tokval: {tokval}, from_pos: {code[char_pos:char_pos+len(tokval)]}")
                
                files_with_tokens.append({
                    'file': path.strip(),
                    'token_count': len(file_tokens),
                    'tokens': file_tokens
                })

            except Exception as e:
                print(f"Tokenization error: {e}")

        return files_with_tokens

    def get_data(self):
        token_path = self.outfile
        if os.path.exists(token_path):            
            with open(token_path, 'r') as f:
                files_with_tokens = json.load(f)

            total_tokens = sum(f['token_count'] for f in files_with_tokens)
            print(f"Loaded {len(files_with_tokens)} files with {total_tokens} total tokens from {token_path}")

        else:            
            files_with_tokens = self.tokenize_data()
            
            # Save to JSON
            with open(token_path, 'w') as f:
                json.dump(files_with_tokens, f, indent=2)

            total_tokens = sum(f['token_count'] for f in files_with_tokens)
            print(f"Saved {len(files_with_tokens)} files with {total_tokens} total tokens to {token_path}")

        return files_with_tokens

# Sample Generation

In [107]:
import random 

TRIGGER_OPERATORS = {
    '=', '==', '!=', '<', '>', '<=', '>=',
    '+', '-', '*', '/', '//', '%', '**',
    '+=', '-=', '*=', '/=',
    'and', 'or', 'not', 'in', 'is',
}

TRIGGER_STRUCTURAL = {
    '(', '[', '{',  # Opening brackets
    ',',            # Comma (in function calls, lists, etc.)
    '.',            # Dot (for attribute access)
    ':',            # Colon (after if/for/def, or type hints)
    '->',           # Type hint arrow
}

CLOSING_BRACES = {
    ')', ']', '}',
}

VALID_TRIGGERS = TRIGGER_STRUCTURAL | TRIGGER_OPERATORS

class SampleGenerator:
    """Generates prediction samples from tokenized files."""
    
    def __init__(self, basedir, samples_file, 
                 n_before=10, n_after=10,
                 valid_triggers=None, one_per_file=False, seed=42):
        self.basedir = basedir
        self.samples_file = samples_file
        self.n_before = n_before
        self.n_after = n_after
        self.valid_triggers = valid_triggers if valid_triggers else VALID_TRIGGERS
        self.one_per_file = one_per_file
        self.seed = seed
    
    def create_samples(self, files_with_tokens):

        # if self.one_per_file:
        #     random.seed(self.seed)
        
        samples = []

        for file_data in files_with_tokens:
            file_path = file_data['file']
            code = open(os.path.join(self.basedir, file_path.strip())).read()
            tokens = file_data['tokens']
            
            # Skip files with too few tokens
            min_tokens_needed = self.n_before + self.n_after + 1
            if len(tokens) < min_tokens_needed:
                continue
            
            # Find all valid indices where trigger token is valid
            valid_token_idxs = []
            for i in range(self.n_before, len(tokens) - self.n_after):
                # The trigger is the token just before the target (at index i-1)
                trigger_token = tokens[i - 1]['value']
                target_token = tokens[i]['value']
                
                if ((trigger_token in self.valid_triggers) and 
                    (target_token not in self.valid_triggers | CLOSING_BRACES)):
                    valid_token_idxs.append(i)
            
            # Skip files with no valid trigger positions
            if len(valid_token_idxs) == 0:                
                continue            
            
            # Select indices based on one_per_file setting
            if self.one_per_file:
                selected_token_idxs = [random.choice(valid_token_idxs)]
            else:
                selected_token_idxs = valid_token_idxs
            
            # Create samples for selected indices
            for target_idx in selected_token_idxs:
                
                # Get context before
                before_start = tokens[target_idx - self.n_before]["start_pos"]
                before_end = tokens[target_idx]["start_pos"]
                code_before = code[before_start:before_end]

                # Get context after                
                after_start = tokens[target_idx]["start_pos"] + len(tokens[target_idx]["value"])
                after_end = tokens[target_idx + self.n_after]["start_pos"] + len(tokens[target_idx]["value"])
                code_after = code[after_start:after_end]
                
                # Target token
                target_token = tokens[target_idx]['value']
                target_start_pos = tokens[target_idx]['start_pos']                
                
                # The trigger is the last token in code_before
                trigger_token = tokens[target_idx-1]['value']
                
                samples.append({
                    'file': file_path,
                    'trigger_token': trigger_token,
                    'target_token': target_token,
                    'target_start_pos': target_start_pos,                                    
                    'code_before': code_before,
                    'code_after': code_after
                })
        
        print(f"Created {len(samples)} samples")
        if self.one_per_file:
            print(f"Mode: One sample per file (randomly selected)")
        else:
            print(f"Mode: All valid positions")
        
        return samples
    
    def get_samples(self, files_with_tokens):
        """Load or create prediction samples."""
        samples_path = self.samples_file
        
        if os.path.exists(samples_path):
            print(f"Loading samples from cache: {samples_path}")
            with open(samples_path, 'r') as f:
                samples = json.load(f)
            print(f"Loaded {len(samples)} samples")
        else:
            print("Creating samples...")
            samples = self.create_samples(files_with_tokens)
            
            # Save samples
            print(f"\nSaving {len(samples)} samples to {samples_path}")
            with open(samples_path, 'w') as f:
                json.dump(samples, f, indent=2)
            print("Samples saved successfully!")
        
        return samples

# Evaluator Class

In [ ]:
class CompletionEvaluator:
    def __init__(self, model: LocalCodeModel, lsp: LSPClient, basedir: str):
        self.model = model
        self.lsp = lsp
        self.basedir = basedir
    
    def filter_completions_by_prob(self, completions, prob_threshold: float=0.8):
        filtered = []
        for completion, prob in completions:
            if prob > prob_threshold:
                filtered.append((completion, prob))
        return filtered

    def filter_completions_by_char(self, completions):
        STOP_CHARS = ['(', ')', '[', ']', '{', '}', ',', ':', ';', '.', "\"", "\'"]
        filtered = []
        for completion, prob in completions:
            if completion not in STOP_CHARS:
                filtered.append((completion, prob))
        return filtered
    
    def filter_by_correctness(self, completions, code_before: str, code_after: str):
        filtered = []
        for completion, prob in completions:            
            modified_code = code_before + completion + code_after                
            is_valid = self.lsp.validate_code(modified_code)
            
            if is_valid:
                filtered.append((completion, prob))
                
        return filtered
        
    def evaluate(self, samples, n: int, prob_threshold: float=0.8):

        if len(samples) == 0:
            raise ValueError("no samples provided")
        if len(samples) < n:
            raise ValueError(f"Not enough samples provided. Must provide at least {n}")
        
        n_correct = 0
        n_total = 0
        i = 0

        while n_total < n:
            if i >= len(samples):
                print(f"Terminated early. Only evaluating over {n_total} samples")
                break

            sample = samples[i]
            code_before, code_after = sample['code_before'], sample['code_after']            
            label = sample['target_token']
            
            print(f"\n--- Sample {i+1}, Valid Sample {n_total+1} ---")
            print(f"Before: {repr(code_before[len(code_before)-20:])}")
            print(f"After: {repr(code_after[:20])})")                       
            print(f"Expected: {label}")
            
            
            start_time = time.perf_counter()
            completions = self.model.generate(code_before=code_before, code_after=code_after)
            end_time = time.perf_counter()
            print("latency_ms:", (end_time - start_time) * 1000)

            print(f"LLM completions: {completions}")
            
            filter_start_time = time.perf_counter()
            filtered = self.filter_completions_by_prob(completions=completions, prob_threshold=prob_threshold)
            filtered = self.filter_completions_by_char(completions=filtered)
            filtered = self.filter_by_correctness(completions=filtered, code_before=code_before, code_after=code_after)
            filter_end_time = time.perf_counter()
            print("filter latency_ms:", (filter_end_time - filter_start_time) * 1000)

            if not filtered:
                print(f"No completions above {prob_threshold:.1%} threshold. Skipping sample...")
                i += 1
                continue

            found_match = False
            matching_prob = 0.0

            for completion, prob in filtered:

                # check filled code with LSP - might not be necessary
                # modified_code = code_before + completion + code_after                
                # is_valid = self.lsp.validate_code(modified_code)

                if completion == label:
                    found_match = True
                    matching_prob = prob
                    break
            
            if found_match:
                print(f"Correct! Pred:`{completion}` ({matching_prob:.1%})")
                n_correct += 1
            else:
                best_pred, best_prob = filtered[0]
                print(f"Incorrect! Pred: `{best_pred}` ({best_prob:.1%}) Label: `{label}`")
            
            n_total += 1
            i += 1
        
        acc = n_correct / n_total
        print(f"\nAccuracy over {n_total} samples: {acc:.1%} (Coverage: {(n_total/i):.1%}, Prob TH: {prob_threshold:.1%})")
        return acc


# Main

In [109]:
import random
random.seed(20)

MODEL_PATH = "./models/qwen2.5-coder-3b-instruct-q4_k_m.gguf"
# MODEL_PATH = "./models/deepseek-coder-1.3b-instruct.Q4_K_M.gguf"
BASE_DIR = "./data/py150_files"
INPUT_FILE = "python100_eval.txt"
OUTPUT_FILE = "./data/eval_tokens.json"
SAMPLES_FILE = "./data/samples.json"

STOP_TOKENS = [
    '(', ')', '[', ']', '{', '}',  # Brackets
    ',', ':', ';',                   # Delimiters  
    '.', 
    '+', '-', '*', '/', '%', '@',   # Arithmetic
    '=', '<', '>', '!',              # Comparison/Assignment
    '&', '|', '^', '~',              # Bitwise
    '\n', '\t', ' ',
    "<|endoftext|>"
]

dataloader = DataLoader(basedir=BASE_DIR, infile=INPUT_FILE, outfile=OUTPUT_FILE)
data = dataloader.get_data()

sample_generator = SampleGenerator(basedir=BASE_DIR, samples_file=SAMPLES_FILE, n_before=100, n_after=100, one_per_file=False)
samples = sample_generator.get_samples(data)
random.shuffle(samples)

model = LocalCodeModel(model_path=MODEL_PATH,
                       max_tokens=10,
                       n_threads=8,
                       n_gpu_layers=-1,
                       stop_tokens=STOP_TOKENS,
                       n_batch=512)

lsp = LSPClient(cmd=['pylsp']) # start python language server

evaluator = CompletionEvaluator(model=model, lsp=lsp, basedir=BASE_DIR)

n_samples = 100
accuracy = evaluator.evaluate(samples=samples, prob_threshold=0.8, n=n_samples)

Loaded 98 files with 85376 total tokens from ./data/eval_tokens.json
Loading samples from cache: ./data/samples.json
Loaded 24741 samples


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64 

Token budget: 1998 tokens (~4995 chars) available for input

--- Sample 1, Valid Sample 1 ---
Before: ' m["out"]["format"].'
After: '() )\n\t\tself.assertEq')
Expected: hash
latency_ms: 599.5839997194707
LLM completions: [('hash', 0.9789388207753184)]
filter latency_ms: 510.13804180547595
Correct! Pred:`hash` (97.9%)

--- Sample 2, Valid Sample 2 ---
Before: ' elif hasattr(self, '
After: '):\n            genre')
Expected: "arxiv"
latency_ms: 1620.0309582054615
LLM completions: [('"arxiv_id"', 0.9338810453067274)]
filter latency_ms: 506.33891578763723
Incorrect! Pred: `"arxiv_id"` (93.4%) Label: `"arxiv"`

--- Sample 3, Valid Sample 3 ---
Before: '_,\n            self.'
After: '()\n        )\n\n\nclass')
Expected: to_header
latency_ms: 1314.2638746649027
LLM completions: [('to_header', 0.9200420954028997)]
filter latency_ms: 506.62395916879177
Correct! Pred:`to_header` (92.0%)

--- Sample 4, Valid Sample 4 ---
Before: '.__parent.ancestor( '
After: '.ScriptNode ) ) :\n\t\t')
Expected: Ga